## Import and define

In [1]:
import sys
cur_path = "/home/vmagent/app"
sys.path.append(cur_path)

from near_dedup import *

JAVA_HOME is not set, use default value of /usr/lib/jvm/java-8-openjdk-amd64/


/usr/local/lib/python3.10/dist-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


Will assign 48 cores and 308492 M memory for spark


23/08/30 16:59:52 WARN Utils: Your hostname, sr414 resolves to a loopback address: 127.0.1.1; using 10.1.2.14 instead (on interface enp134s0f1)
23/08/30 16:59:52 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/08/30 16:59:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


per core memory size is 6.276 GB and shuffle_disk maximum capacity is 8589934592.000 GB


## Configurate DIR

In [ ]:
rdp = SparkDataProcessor()
spark=rdp.spark 

In [2]:
data_files = get_data_files('/home/vmagent/app/PILE/EuroParl')
dup_dir = "/home/vmagent/app/PILE_output/EuroParl/deduplicate"
ngram_size = 13
num_perm = 256
bands = 9
ranges = 13

## Load data into Spark

In [3]:
with Timer("Load data with RowID"):
    df = read_json(data_files, spark).cache()
    total_length = df.count()

Load data with RowID started ...
/home/vmagent/app/PILE/EuroParl/EuroParliamentProceedings_1996_2011.jsonl


Load data with RowID took 16.729979965020902 sec


## Get minHashLSH edges

In [4]:
pipeline = minHashLSH_prepare(df, num_perm, ngram_size, bands, ranges)
with Timer("generate minHashLsh"):
    if os.path.exists(dup_dir):
        shutil.rmtree(dup_dir, ignore_errors=True)
    results = pipeline.saveAsTextFile(dup_dir)

num_bands is 9, ranges is 13
generate minHashLsh started ...


generate minHashLsh took 271.9689698460279 sec


In [4]:
spark.stop()

## Generate connected components

In [3]:
with Timer(f"generate_connected_components all"):
    dup_connected_args = argparse.Namespace()
    dup_connected_args.input_dir = dup_dir
    dup_connected_args.out_file = os.path.join(
        dup_dir, "connected_components.pickle"
    )
    generate_connected_components.generate_connected_components_mp(dup_connected_args)

generate_connected_components all started ...
Started graph building


loop on file: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 203/203 [00:00<00:00, 6960.22it/s]


Failed to process /home/vmagent/app/PILE_output/EuroParl/deduplicate/duplicates.pickle, error is 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte
Failed to process /home/vmagent/app/PILE_output/EuroParl/deduplicate/connected_components.pickle, error is 'utf-8' codec can't decode byte 0x80 in position 0: invalid start byte
length of the set of duplicates: 902 0.034215688705444336


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 902/902 [00:00<00:00, 861594.67it/s]

number of connected components: 340 0.03926563262939453
Graph generated duplicates list!!! 0.041168928146362305
generate_connected_components all took 0.045084498007781804 sec


##  convert as duplicates dict

In [6]:
with Timer(f"generate_duplicates_dict all"):
    dup_docs = os.path.join(dup_dir, "duplicates.pickle")
    dup_dict_args = argparse.Namespace()
    dup_dict_args.input_file = os.path.join(
        dup_dir, "connected_components.pickle"
    )
    dup_dict_args.out_file = dup_docs
    generate_duplicates_dict.generate_duplicates(dup_dict_args)

generate_duplicates_dict all started ...
Processing duplicates!!!


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 340/340 [00:00<00:00, 175970.31it/s]


number of duplicate documents that will be removed: 886
generate_duplicates_dict all took 0.008639081963337958 sec


## View result

In [7]:
dup_dict = pickle.load(open(os.path.join(dup_dir, "duplicates.pickle"), 'rb'))
dup_sum = 0
for _, v in dup_dict.items():
    dup_sum += len(list(v))

print(f"Completed!!")
print(f"    total processed {total_length} documents")
print(f"    total detected {dup_sum} duplicated documents")
print(f"    duplicate ratio is {dup_sum/total_length}")

Completed!!
    total processed 69814 documents
    total detected 886 duplicated documents
    duplicate ratio is 0.01269086429655943
